In [3]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import mixture
sns.set(style='whitegrid')
sns.set_palette('bright')
import warnings
warnings.filterwarnings('ignore')
import collections
from scipy.spatial import distance


In [4]:
df = pd.read_csv('preci_delhi_clean.csv')
del df["Unnamed: 0"]
df.date = pd.to_datetime(df.date,format = '%Y-%m-%d')
df

,date,value
0,2000-06-01,33.405911
1,2000-06-02,0.022229
2,2000-06-03,25.713945
3,2000-06-04,11.130651
4,2000-06-05,0.006525
...,...,...
7400,2020-09-04,0.481709
7401,2020-09-05,45.213516
7402,2020-09-06,0.192533
7403,2020-09-07,23.122570


In [5]:
df19 = df[df.date.dt.year==2019]

In [6]:
def gmm2k(q1,q2):
    lowest_bic = np.infty
    bic = []
    n_components_range = range(1, 8)
    cv_types = ['spherical', 'tied', 'diag', 'full']
    for cv_type in cv_types:
        for n_components in n_components_range:
            # Fit a Gaussian mixture with EM
            gmm = mixture.GaussianMixture(n_components=n_components,
                                          covariance_type=cv_type)
            q1 = np.asarray(q1)
            q1 =q1.reshape(-1,1)
            gmm.fit(q1)
            bic.append(gmm.bic(q1))
            if bic[-1] < lowest_bic:
                lowest_bic = bic[-1]
                best_gmmq1 = gmm
    lowest_bic = np.infty
    bic = []
    n_components_range = range(1, 8)
    cv_types = ['spherical', 'tied', 'diag', 'full']
    for cv_type in cv_types:
        for n_components in n_components_range:
            # Fit a Gaussian mixture with EM
            gmm = mixture.GaussianMixture(n_components=n_components,
                                          covariance_type=cv_type)
            q2 = np.asarray(q2)
            q2 =q2.reshape(-1,1)
            gmm.fit(q2)
            bic.append(gmm.bic(q2))
            if bic[-1] < lowest_bic:
                lowest_bic = bic[-1]
                best_gmmq2 = gmm
    mins = min(best_gmmq1.n_components,best_gmmq2.n_components)
    if best_gmmq1.n_components > best_gmmq2.n_components:
        lowest_bic = np.infty
        bic = []
        cv_types = ['spherical', 'tied', 'diag', 'full']
        for cv_type in cv_types:
            # Fit a Gaussian mixture with EM
            gmm = mixture.GaussianMixture(n_components=mins,
                                              covariance_type=cv_type)
            q1 = np.asarray(q1)
            q1 =q1.reshape(-1,1)
            gmm.fit(q1)
            bic.append(gmm.bic(q1))
            if bic[-1] < lowest_bic:
                lowest_bic = bic[-1]
                best_gmmq1 = gmm
    else:
        lowest_bic = np.infty
        bic = []
        cv_types = ['spherical', 'tied', 'diag', 'full']
        for cv_type in cv_types:
            # Fit a Gaussian mixture with EM
            gmm = mixture.GaussianMixture(n_components=mins,
                                              covariance_type=cv_type)
            q2 = np.asarray(q2)
            q2 =q2.reshape(-1,1)
            gmm.fit(q2)
            bic.append(gmm.bic(q2))
            if bic[-1] < lowest_bic:
                lowest_bic = bic[-1]
                best_gmmq2 = gmm
    #print(best_gmmq1.get_params())
    #print(best_gmmq2.get_params())
    #print(best_gmmq1.means_)
    #print(best_gmmq2.means_)
    #print(best_gmmq1.weights_)
    #print(best_gmmq2.weights_)
    means=[]
    for tt in range(len(best_gmmq1.means_)):
        means.append(best_gmmq1.means_[tt][0])
    x = dict(zip(best_gmmq1.weights_,means))
    od1 = collections.OrderedDict(sorted(x.items(),reverse=True))
    od1 = list(zip(list(od1.keys()),list(od1.values())))
    means=[]
    for tt in range(len(best_gmmq2.means_)):
        means.append(best_gmmq2.means_[tt][0])
    y = dict(zip(best_gmmq2.weights_,means))
    od2 = collections.OrderedDict(sorted(y.items(),reverse=True))
    od2 = list(zip(list(od2.keys()),list(od2.values())))
    #print(od1)
    #print(od2)
    diffmean = []
    diffweights = []
    for tt in range(len(od2)):
        diffmean.append(od1[tt][1] - od2[tt][1])
        diffweights.append(od1[tt][0] - od2[tt][0]) 
    print(diffweights)
    r=[]
    for i in range(0,len(diffmean)):
        r.append(diffmean[i] * diffweights[i] * 10000)
    return od1,od2,sum(r)

In [7]:
def gmm10k(q1,q2):
    lowest_bic = np.infty
    bic = []
    n_components_range = range(1, 8)
    cv_types = ['spherical', 'tied', 'diag', 'full']
    for cv_type in cv_types:
        for n_components in n_components_range:
            # Fit a Gaussian mixture with EM
            gmm = mixture.GaussianMixture(n_components=n_components,
                                          covariance_type=cv_type)
            q1 = np.asarray(q1)
            q1 =q1.reshape(-1,1)
            gmm.fit(q1)
            bic.append(gmm.bic(q1))
            if bic[-1] < lowest_bic:
                lowest_bic = bic[-1]
                best_gmmq1 = gmm
    lowest_bic = np.infty
    bic = []
    n_components_range = range(1, 8)
    cv_types = ['spherical', 'tied', 'diag', 'full']
    for cv_type in cv_types:
        for n_components in n_components_range:
            # Fit a Gaussian mixture with EM
            gmm = mixture.GaussianMixture(n_components=n_components,
                                          covariance_type=cv_type)
            q2 = np.asarray(q2)
            q2 =q2.reshape(-1,1)
            gmm.fit(q2)
            bic.append(gmm.bic(q2))
            if bic[-1] < lowest_bic:
                lowest_bic = bic[-1]
                best_gmmq2 = gmm
    
    #print(best_gmmq1.get_params())
    #print(best_gmmq2.get_params())
    #print(best_gmmq1.means_)
    #print(best_gmmq2.means_)
    #print(best_gmmq1.weights_)
    #print(best_gmmq2.weights_)
    means=[]
    for tt in range(len(best_gmmq1.means_)):
        means.append(best_gmmq1.means_[tt][0])
    x = dict(zip(best_gmmq1.weights_,means))
    od1 = collections.OrderedDict(sorted(x.items(),reverse=True))
    od1 = list(zip(list(od1.keys()),list(od1.values())))
    means=[]
    for tt in range(len(best_gmmq2.means_)):
        means.append(best_gmmq2.means_[tt][0])
    y = dict(zip(best_gmmq2.weights_,means))
    od2 = collections.OrderedDict(sorted(y.items(),reverse=True))
    od2 = list(zip(list(od2.keys()),list(od2.values())))
    maxx = max(len(od1),len(od2))
    diff = abs(len(od1) - len(od2))
    if len(od2) > len(od1):
        for i in range(0,diff):
            od1.append((0,0))
    elif len(od2) < len(od1):
        for i in range(0,diff):
            od2.append((0,0))
    #print(maxx)
    #print(od1)
    #print(od2)
    diffmean = []
    diffweights = []
    for tt in range(len(od2)):
        diffmean.append(od1[tt][1] - od2[tt][1])
        diffweights.append(od1[tt][0] - od2[tt][0]) 
    #print(diffweights)
    r=[]
    for i in range(0,len(diffmean)):
        r.append(diffmean[i] * diffweights[i])
    #print(len(od1) , len(od2))
    return od1,od2,sum(r)

In [8]:
def gmm20k(q1,q2):
    lowest_bic = np.infty
    bic = []
    n_components_range = range(1, 8)
    cv_types = ['spherical', 'tied', 'diag', 'full']
    for cv_type in cv_types:
        for n_components in n_components_range:
            # Fit a Gaussian mixture with EM
            gmm = mixture.GaussianMixture(n_components=n_components,
                                          covariance_type=cv_type)
            q1 = np.asarray(q1)
            q1 =q1.reshape(-1,1)
            gmm.fit(q1)
            bic.append(gmm.bic(q1))
            if bic[-1] < lowest_bic:
                lowest_bic = bic[-1]
                best_gmmq1 = gmm
    lowest_bic = np.infty
    bic = []
    n_components_range = range(1, 8)
    cv_types = ['spherical', 'tied', 'diag', 'full']
    for cv_type in cv_types:
        for n_components in n_components_range:
            # Fit a Gaussian mixture with EM
            gmm = mixture.GaussianMixture(n_components=n_components,
                                          covariance_type=cv_type)
            q2 = np.asarray(q2)
            q2 =q2.reshape(-1,1)
            gmm.fit(q2)
            bic.append(gmm.bic(q2))
            if bic[-1] < lowest_bic:
                lowest_bic = bic[-1]
                best_gmmq2 = gmm
    
    means=[]
    for tt in range(len(best_gmmq1.means_)):
        means.append(best_gmmq1.means_[tt][0])
    x = dict(zip(means,best_gmmq1.weights_))
    od1 = collections.OrderedDict(sorted(x.items(),reverse=True))
    od1 = list(zip(list(od1.keys()),list(od1.values())))
    #print(od1)
    means=[]
    for tt in range(len(best_gmmq2.means_)):
        means.append(best_gmmq2.means_[tt][0])
    y = dict(zip(means,best_gmmq2.weights_))
    od2 = collections.OrderedDict(sorted(y.items(),reverse=True))
    od2 = list(zip(list(od2.keys()),list(od2.values())))
    #print(od2)
    maxx = max(len(od1),len(od2))
    diff = abs(len(od1) - len(od2))
    if len(od2) > len(od1):
        for i in range(0,diff):
            od1.append((0,0))
    elif len(od2) < len(od1):
        for i in range(0,diff):
            od2.append((0,0))

    diffmean = []
    diffweights = []
    for tt in range(len(od2)):
        diffmean.append(abs(od1[tt][1] - od2[tt][1]))
        diffweights.append(abs(od1[tt][0] - od2[tt][0])) 
    #print(diffweights)
    r=[]
    for i in range(0,len(diffmean)):
        r.append(diffmean[i] * diffweights[i])
    #print(len(od1) , len(od2))
    return od1,od2,sum(r)

In [9]:
df19j = df19[df19.date.dt.month == 7]
df19f = df19[df19.date.dt.month == 6]
qw,we,r= gmm10k(df19j.value,df19f.value)
print(qw)
print(we)
print(r)

[(0.3712574401381559, 0.24218771268214254), (0.2738966585264367, 3.3668455122183314), (0.1711468679094593, 23.90058322702762), (0.08692483987830626, 14.699361528422827), (0.0322580645161291, 53.0784726015751), (0.03225806451538409, 41.866538684661585)]
[(0.5309985038443144, 0.00039152551421262505), (0.2690014961556857, 0.08997976193206099), (0.06666666666666674, 1.5667395658895926), (0.03333333333333341, 4.171361481409863), (0, 0), (0, 0)]
5.937814172967082


In [10]:
df19j = df19[df19.date.dt.month == 7]
df19f = df19[df19.date.dt.month == 6]
qw,we,r= gmm2k(df19j.value,df19f.value)
print(qw)
print(we)
print(r)

[-0.15974154259767542, 0.004895641262268158, 0.1044802012427925, 0.0535915065449729]
[(0.3712574401381559, 0.24218771268214254), (0.2738966585264367, 3.366845512218332), (0.17114686790945924, 23.90058322702762), (0.0869248398783063, 14.699361528422829), (0.0322580645161291, 82.12354765421324), (0.03225806451538409, 41.866538684661585)]
[(0.5309989827358313, 0.00039152568004190364), (0.26900101726416853, 0.08997992109499545), (0.06666666666666674, 1.5667395658895926), (0.03333333333333341, 3.408147138067614)]
29159.751326350397


In [11]:
a = np.zeros(shape=(12, 12))
for i in range(1,13):
    for j in range(1,13):
        dfc = df19[df19.date.dt.month == i]
        dfd = df19[df19.date.dt.month == j]
        _,_,r = gmm10k(dfc.value,dfd.value)
        a[i-1][j-1] = r
a

array([[ 0.00000000e+00, -2.01581334e-01,  2.62858049e-01,
         1.61619562e+00, -1.23284018e+00,  1.14295825e-01,
         1.95073213e+00,  1.01289930e+00,  3.44527586e-01,
        -7.75435506e-03, -3.01034238e-02,  2.08166204e+00],
       [-2.01581334e-01,  7.06605081e-02,  1.79553944e-01,
         1.18973427e+00, -6.28454346e-02,  3.46894537e-01,
         4.41258010e-01,  1.73991093e+00, -7.81876066e-01,
         6.39702988e-01, -1.02757800e-01,  4.43265356e-01],
       [ 2.62858049e-01,  7.82961413e-01,  0.00000000e+00,
         8.99924356e-03,  2.39850323e-01, -6.15809356e-03,
         6.71923694e+00,  5.29883057e+00,  5.09678849e-01,
        -1.29875600e-02, -5.40525730e-02,  8.30317578e-02],
       [ 1.61619562e+00,  1.16690822e+00,  8.99924356e-03,
         7.34859391e-01,  2.34006767e-01,  1.46643416e-02,
         6.85621553e+00,  5.09294529e+00,  7.36329527e-01,
         3.62269414e-02, -6.41413170e-03, -1.57530012e-02],
       [-1.41549990e+00, -6.28454346e-02,  2.3985032

In [12]:
dfw = pd.DataFrame(a)
cm = sns.light_palette("green", as_cmap=True)
s = dfw.style.background_gradient(cmap=cm)
s

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.000000,-0.201581,0.262858,1.616196,-1.232840,0.114296,1.950732,1.012899,0.344528,-0.007754,-0.030103,2.081662
1,-0.201581,0.070661,0.179554,1.189734,-0.062845,0.346895,0.441258,1.739911,-0.781876,0.639703,-0.102758,0.443265
2,0.262858,0.782961,0.000000,0.008999,0.239850,-0.006158,6.719237,5.298831,0.509679,-0.012988,-0.054053,0.083032
3,1.616196,1.166908,0.008999,0.734859,0.234007,0.014664,6.856216,5.092945,0.736330,0.036227,-0.006414,-0.015753
4,-1.415500,-0.062845,0.239850,0.234007,0.000000,0.140990,2.894230,0.981131,-1.039183,0.099348,0.117257,0.337807
5,0.112130,-0.278195,-0.043707,0.233969,0.117191,0.000000,6.915654,4.559825,0.464266,-0.154162,-0.018658,0.184907
6,1.950732,2.650914,7.209480,6.856216,2.894230,6.660174,-3.992806,-4.630143,1.722648,6.350492,6.359115,9.033281
7,1.474973,1.739903,5.193109,4.803938,0.981123,4.559825,0.104652,-0.000000,0.574021,4.554952,3.989766,5.393259
8,0.344528,-0.309779,0.616267,0.076453,-0.594216,0.463079,2.880922,0.574021,-3.054471,-0.168122,0.281759,0.347543
9,-0.007754,0.639703,-0.012988,0.225574,0.099348,-0.154162,6.528328,4.554952,-0.168122,0.000000,-0.321705,0.111691


In [13]:
s

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.000000,-0.201581,0.262858,1.616196,-1.232840,0.114296,1.950732,1.012899,0.344528,-0.007754,-0.030103,2.081662
1,-0.201581,0.070661,0.179554,1.189734,-0.062845,0.346895,0.441258,1.739911,-0.781876,0.639703,-0.102758,0.443265
2,0.262858,0.782961,0.000000,0.008999,0.239850,-0.006158,6.719237,5.298831,0.509679,-0.012988,-0.054053,0.083032
3,1.616196,1.166908,0.008999,0.734859,0.234007,0.014664,6.856216,5.092945,0.736330,0.036227,-0.006414,-0.015753
4,-1.415500,-0.062845,0.239850,0.234007,0.000000,0.140990,2.894230,0.981131,-1.039183,0.099348,0.117257,0.337807
5,0.112130,-0.278195,-0.043707,0.233969,0.117191,0.000000,6.915654,4.559825,0.464266,-0.154162,-0.018658,0.184907
6,1.950732,2.650914,7.209480,6.856216,2.894230,6.660174,-3.992806,-4.630143,1.722648,6.350492,6.359115,9.033281
7,1.474973,1.739903,5.193109,4.803938,0.981123,4.559825,0.104652,-0.000000,0.574021,4.554952,3.989766,5.393259
8,0.344528,-0.309779,0.616267,0.076453,-0.594216,0.463079,2.880922,0.574021,-3.054471,-0.168122,0.281759,0.347543
9,-0.007754,0.639703,-0.012988,0.225574,0.099348,-0.154162,6.528328,4.554952,-0.168122,0.000000,-0.321705,0.111691


In [26]:
df19j = df19[df19.date.dt.month == 7]
df19f = df19[df19.date.dt.month == 6]
qw,we,r= gmm20k(df19j.value,df19f.value)
print(qw)
print(we)
print(r)

[(0.03680788960154117, 0.06764230961801905), (0.009093045896577728, 0.9323576903819809), (0, 0)]
[(0.05143587184378962, 0.0667888273364624), (0.008505409622636169, 0.29050665528132086), (0.0017537042888375552, 0.6427045173822166)]
0.0015167733429392338


In [27]:
a = np.zeros(shape=(12, 12))
for i in range(1,13):
    for j in range(1,13):
        dfc = df19[df19.date.dt.month == i]
        dfd = df19[df19.date.dt.month == j]
        _,_,r = gmm20k(dfc.value,dfd.value)
        a[i-1][j-1] = r
a

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 6.49000000e-02, 4.27960902e-02, 9.53870969e-02,
        1.51632334e-02, 1.06940969e-02, 4.56631087e-03, 3.50548272e-03],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 5.15985221e-02, 4.28184363e-02, 9.53870969e-02,
        1.52422568e-02, 3.46086869e-02, 4.56631087e-03, 3.50548272e-03],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 5.15985221e-02, 4.27960902e-02, 9.53870969e-02,
        1.52422568e-02, 3.46086869e-02, 4.56631087e-03, 3.50548272e-03],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 5.12563446e-02, 4.24542257e-02, 9.50440861e-02,
        1.50240636e-02, 1.02116570e-02, 4.23617930e-03, 3.17258090e-03],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 5.15985221e-02, 4.29903183e-02, 9.53

In [28]:
dfw = pd.DataFrame(a)
cm = sns.light_palette("green", as_cmap=True)
s = dfw.style.background_gradient(cmap=cm)
s

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.064900,0.042796,0.095387,0.015163,0.010694,0.004566,0.003505
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.051599,0.042818,0.095387,0.015242,0.034609,0.004566,0.003505
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.051599,0.042796,0.095387,0.015242,0.034609,0.004566,0.003505
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.051256,0.042454,0.095044,0.015024,0.010212,0.004236,0.003173
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.051599,0.042990,0.095387,0.015163,0.034609,0.004566,0.003505
5,0.051599,0.064900,0.051599,0.051256,0.051599,0.016320,0.001517,0.000986,0.039488,0.001075,0.007793,0.007490
6,0.042796,0.042990,0.042796,0.042477,0.042796,0.001517,0.000000,0.007249,0.006896,0.008208,0.001343,0.001063
7,0.095387,0.095387,0.095387,0.095044,0.095387,0.000986,0.007249,0.000000,0.029055,0.002930,0.014621,0.014096
8,0.015242,0.015163,0.015242,0.014946,0.015163,0.012647,0.006970,0.029055,0.000000,0.008730,0.006283,0.006818
9,0.010449,0.034609,0.034609,0.034254,0.034609,0.001075,0.005578,0.002930,0.000509,0.000000,0.012779,0.012717


## OrderbyMeans

In [10]:
means=[]
for i in range(len(best_gmmq1.means_)):
    means.append(best_gmmq1.means_[i][0])
x = dict(zip(means,best_gmmq1.weights_))
od1 = collections.OrderedDict(sorted(x.items(),reverse=True))
od1

NameError: name 'best_gmmq1' is not defined

In [ ]:
means=[]
for i in range(len(best_gmmq2.means_)):
    means.append(best_gmmq2.means_[i][0])
x = dict(zip(means,best_gmmq2.weights_))
od2 = collections.OrderedDict(sorted(x.items(),reverse=True))
od2

## OrderbyWeights

In [ ]:
means=[]
for i in range(len(best_gmmq1.means_)):
    means.append(best_gmmq1.means_[i][0])
x = dict(zip(best_gmmq1.weights_,means))
od1 = collections.OrderedDict(sorted(x.items(),reverse=True))
od1 = list(zip(list(od1.keys()),list(od1.values())))
od1

In [ ]:
means=[]
for i in range(len(best_gmmq2.means_)):
    means.append(best_gmmq2.means_[i][0])
y = dict(zip(best_gmmq2.weights_,means))
od2 = collections.OrderedDict(sorted(y.items(),reverse=True))
od2 = list(zip(list(od2.keys()),list(od2.values())))
od2

In [ ]:
thres = 0.05
diffmean = []
diffweights = []
for i in range(len(od1)):
    diffmean.append(od1[i][1] - od2[i][1])
    diffweights.append(od1[i][0] - od2[i][0])        

In [ ]:
print(diffmean)
print(diffweights)